In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

import time
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager

## Version with only Distillery and Year 
#### (for remove duplicates)

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://www.whisky.com/whisky-database/bottle-search/expert-taste-search/fdb/Bottles.html?w_age_min=5&w_video=1&w_image=1&w_ratingCount_min=10&tx_datamintsflaschendb_pi4%5BresultListLayout%5D=0&cHash=0e639d5b51424cd1a47227d596f3701c#dm_flaschendb_flasche_search_sorting')

driver.maximize_window()

click_agreement = driver.find_element_by_xpath("//a[@title='Accept']")
click_agreement.click()

time.sleep(3)
previous_height = driver.execute_script('return document.body.scrollHeight')

while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(3)

    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == previous_height:
        break
    previous_height = new_height

name, year, keywords, volume = [],[],[],[]
page = BeautifulSoup (driver.page_source, 'html.parser')

whisky = page.find_all(class_= 'item item-wrap')
top = 0

# select only with rating
for whisky_info in whisky:
    if whisky_info.find(class_='rating-wrap') and top < 500:
        top += 1
    # take name
        name += [whisky_info.find(class_ = 'marke').text]
    # take year
        #if whisky_info.find(class_ = 'alterEtikett'):
        try:
            year += [re.search(r'\d+',(whisky_info.find(class_ = 'alterEtikett').text))[0]]
        except: 
            year += [whisky_info.find(class_ = 'alteEtikett')]

top_whisky = pd.DataFrame({'Name': name, 'Year': year})   
top_whisky.drop_duplicates(inplace = True)
top_whisky = top_whisky[:100]
top_whisky['Name'] = top_whisky['Name'].apply(lambda x :x.strip())
top_whisky.to_csv('Top100_whisky_trim.csv', index = False)




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\user\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


## Version with Distillery, Year, keywords and volume 

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://www.whisky.com/whisky-database/bottle-search/expert-taste-search/fdb/Bottles.html?w_age_min=5&w_video=1&w_image=1&w_ratingCount_min=10&tx_datamintsflaschendb_pi4%5BresultListLayout%5D=0&cHash=0e639d5b51424cd1a47227d596f3701c#dm_flaschendb_flasche_search_sorting')

driver.maximize_window()

# Accept cookie banner
click_agreement = driver.find_element_by_xpath("//a[@title='Accept']")
click_agreement.click()

time.sleep(3)
previous_height = driver.execute_script('return document.body.scrollHeight')

# Scroll to bottom
while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(3)

    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == previous_height:
        break
    previous_height = new_height

# scraping
name, year, keywords, volume = [],[],[],[]
page = BeautifulSoup (driver.page_source, 'html.parser')

whisky = page.find_all(class_= 'item item-wrap')
top = 0

# select only with rating
for whisky_info in whisky:
    if whisky_info.find(class_='rating-wrap') and top < 500:
        top += 1
    # take name
        name += [whisky_info.find(class_ = 'marke').text]
    # take year
        #if whisky_info.find(class_ = 'alterEtikett'):
        try:
            year += [re.search(r'\d+',(whisky_info.find(class_ = 'alterEtikett').text))[0]]
        except: 
            year += [whisky_info.find(class_ = 'alteEtikett')]
    # take keywords if any 
        if whisky_info.find(class_ = 'namenszusatz'):
            keywords += [whisky_info.find(class_ = 'namenszusatz').text]
        else:
            keywords += [whisky_info.find(class_ = 'namenszusatz')]
    # take volume
        if whisky_info.find(class_ = 'flaschengroesse'):
            volume += [float(re.search(r'\d+\.\d+|\d+',whisky_info.find(class_ = 'flaschengroesse').text)[0])*1000]
        else:
            volume += [whisky_info.find(class_ = 'flaschengroesse')]


top_whisky = pd.DataFrame({'Name': name, 'Year': year, 'Keywords': keywords,'Size': volume})    
top_whisky.to_csv('Top100_whisky.csv', index = True)
